## Прогнозирование на несколько шагов вперёд ансамблями над регрессионными деревьями

Регрессионные деревья и ансамбли над ними по умолчанию не предназначены для учёта временной структуры. Однако прогнозирование на несколько разных горизонтов одной и той же моделью, базирующейся на деревьях, можно осуществить следующим образом.

Возьмём обучающую выборку для задачи прогнозирования на один шаг вперёд. При группировке по идентификатору объекта применим операцию сдвига целевой переменной на $k$ шагов, где $k$ принимает значения от 0 до $(h-1)$, а $h$ — желаемый горизонт прогнозирования. Получим выборку, которая в $h$ раз больше по размеру, чем исходная. Добавим в эту выборку признак, принимающий значения от 1 до $h$ и равный $(k+1)$. Этот признак можно интерпретировать как то, на сколько шагов вперёд строится прогноз. Если теперь обучить модель на таких данных, то в зависимости от поданного на стадии предсказания значения нового признака модель сможет предсказывать на любое количество шагов вперёд от 1 до $h$.

## Многозадачное обучение (multi-task learning)

Допустим, есть вспомогательная задача, для которой имеется много размеченных данных, и есть основная задача, для которой размеченных данных существенно меньше. Также допустим, что эти две задачи работают с одними и теми же объектами, но предсказывают на них разные целевые функции. Например, во вспомогательной задаче нужно определить пол человека с портретного фото, а в основной задаче требуется определить цвет волос.

В описанной ситуации можно поступить следующим образом. Построим нейронную сеть для решения вспомогательной задачи и обучим её. Затем уберём сколько-то последних слоёв получившейся сети, и вместо них поставим новые, такие что размерность выходного слоя уже соответствует основной задаче.

Далее есть два варианта:

1) Зафиксировать веса оставленных слоёв нейронной сети, обученной под вспомогательную задачу, и просто прогонять объекты через оставленные слои как через статичный предобработчик;

2) Веса оставшихся от вспомогательной задачи слоёв обучать наравне с весами новых слоёв, то есть считать, что они просто были инициализированы по-другому, но больше отличий между ними и весами новых слоёв нет.

Какой из вариантов выбрать, зависит от числа данных в основной задаче. Если острого недостатка в них нет, то второй способ, как правило, даёт более высокий результат. Первый способ, однако, быстрее, и также он менее требователен к количеству данных (а если размеченных данных совсем мало, то можно даже использовать результат прогона через оставленные обученные слои в качестве признаков для более простого метода: например, метода ближайших соседей).

Новая нейронная сеть обучится лучше, чем если бы сеть с такой же архитектурой обучалась на данных для основной задачи с нуля. Дело в том, что при описанном подходе нейронная сеть будет опираться на скрытое представление, выученное по большому количеству размеченных данных оставленными слоями старой нейронной сети. Также подстройка весов под разные задачи способствует увеличению обобщающей способности. Подробности о том, почему многозадачное обучение работает, есть в разделе 3 статьи [Ruder, 2017](https://arxiv.org/pdf/1706.05098.pdf).

Ещё один подход к многозадачному обучению таков: обучать нейронную сеть параллельно то под вспомогательную задачу, то под основную, при каждом переключении задачи оставляя сколько-то первых слоёв, но заменяя последние слои на то, что было ранее в задаче, к которой происходит переход.

## Как подобрать сложность модели без кросс-валидации?

Иногда в моделях из одного и того же семейства есть параметр, регулирующий сложность. Например, в семействе полиномиальных функций сложностью можно считать максимальную допустимую степень многочлена. Довольно часто сложность можно положить равной количеству оцениваемых по данным параметров — эта формулировка, в частности, включает в себя предыдущую, ведь при оценивании многочленом степени не выше $d$ оценивается $d+1$ параметр.

Модели из одного и того же семейства, отличающиеся друг от друга только сложностью, можно оценивать при помощи информационных критериев.

Информационный критерий Акаике имеет вид:
$$\mathrm{AIC} = 2k - 2l,$$
где $k$ — количество оценённых по данным параметров, а $l$ — значение функционала качества, достигнутое на тренировочной выборке (например, логарифм правдоподобия данных при условии того, что выборка порождается из распределения, задаваемого той моделью, которая была построена на тренировочной выборке).

Байесовский информационный критерий имеет вид:
$$\mathrm{BIC} = \log{(n)} \, k - 2l,$$
где $n$ — размер тренировочной выборки.

По сути, информационные критерии представляют собой оценку качества подгонки под данные с дополнительным штрафом за сложность модели. Интуитивно говоря, чем сложнее модель, тем больше ситуаций, под которые её можно подогнать и, значит, тем менее ценны уровни качества подгонки под данные.

Чем ниже значение какого-либо информационного критерия, тем более хорошей считается модель.

## Откуда может взяться недовольство заказчика машинно-обученной моделью?

Любая прикладная задача моделирования имеет заказчика, неважно, внутреннего или внешнего. Когда заказчик принимает модель, он решает, устраивает ли она его. Причины недовольства заказчика бывают такие:

1) Модель, и в самом деле, плохая;

2) Заказчик не умеет пользоваться моделью;

3) Ожидания заказчика неоправданно завышены.

Какой бы ни была истинная причина, заказчик, скорее всего, решит, что реализовался первый сценарий: модель плохая. Поэтому в интересах специалиста по моделированию избежать ситуаций, когда хорошая модель попадает во второй или третий сценарий.

Разберём эти два сценария подробнее.

Пример сценария, когда моделью не умеют пользоваться, таков: сервис на базе модели рассылает своевременные уведомления о появлении объектов положительного класса, но люди, которым они приходят, игнорируют их. Говоря более общо, можно сказать, что возможны осложнения с интеграцией в бизнес-процессы заказчика. Чтобы упредить их возникновение, полезно задать следующие вопросы:
* Что должно происходить с результатами работы сервиса?
* Как именно сервис приносит конечную ценность?
* Все ли исполнители знают и понимают ответы на два предыдущих вопроса?
* Можно ли оперативно отслеживать сбои в исполнении действий на базе сервиса и можно ли предложить количественные метрики оценки качества исполнения таких действий?
* Все ли признаки, использованные при обучении модели, будут доступны на стадии применения модели вовремя и в сопоставимом качестве?

Проблему завышенных ожиданий проще всего решить, ещё до старта работ по проекту согласовав метрику успеха и её целевой уровень. Сделать это можно, проведя оценку конечного эффекта от внедрения сервиса. Этот конечный эффект может быть как финансовым, то есть выражающимся в деньгах, так и каким-либо более широким (скажем, социальным), но, так или иначе, измеримым.

Впрочем, бывают ситуации, когда заказчик рассуждает не в терминах конечного эффекта, а в терминах всестороннего описания зависимостей — в этом случае заказчик может потребовать, скажем, почти стопроцентную точность по положительному классу. Разумеется, модель, не способная точно описать все зависимости, тоже может быть полезна. Проиллюстрировать это заказчику можно при помощи такого сравнения: если есть лотерея, где вероятность выиграть равна $p$, стоимость участия составляет $c$, а выигрыш приносит $g$, то ответ на вопрос, рационально ли участвовать в такой лотерее, зависит не от того, насколько вероятность $p$ близка к 100%, а от того, как друг с другом соотносятся все три указанных параметра, ведь ожидаемый чистый выигрыш равен $pg - c$.

## Субдискретизация (pooling)

Субдискретизация (pooling) устроена так: по входу слоя проходит фильтр (filter, kernel) и каждый раз от всего, что попадает в него, берёт значение некоторой агрегирующей функции, становящееся значением в соответствующем узле (нейроне) (unit, neuron). Например, в качестве агрегирующей функции можно взять максимум — тогда речь пойдёт о субдискретизации максимумом (max pooling).

Какие желаемые свойства добавляет субдискретизация максимумом?
* Возникает устойчивость к сдвигам изображения на небольшое количество пикселей;
* Появляется возможность реализовать следующую логику: присутствие детектируемого объекта на объединении участков зависит от того, присутствует ли объект хотя бы на одном из этих участков, а не от доли участков, на которых он присутствует.

## Три фундаментальных источника ошибок в машинном обучении

Оптимальный в байесовском смысле регрессор (или классификатор) — это то, что задаётся следующим формальным выражением:
$$f_{opt} = \arg \min_{f \in F} \int_{(x, y) \sim P_{true}(x, y)}l(f(x), y),$$
где $F$ — множество всех возможных регрессоров (или классификаторов соответственно), $P_{true}$ — истинное совместное распределение признаков и целевой переменной, а $l$ — функция потерь, по значению регрессора (классификатора) на векторе признаков и целевой переменной на этом векторе признаков возвращающая соответствующий штраф.

В реальности есть ограничения, не позволяющие найти $f_{opt}$.

Во-первых, неизбежна ошибка приближения (approximation error): поиск проводится не по всему множеству возможных регрессоров (классификаторов), а лишь по какому-то подмножеству $\overline{F} \subset F$, например, по некому параметрическому семейству. Скажем, в случае с линейной регрессией $\overline{F}$ — это множество всех линейных регрессоров. Как следствие, ищется уже нечто другое:
$$\hat{f_1} = \arg \min_{f \in \overline{F}} \int_{(x, y) \sim P_{true}(x, y)}l(f(x), y).$$
Ошибка, вызываемая отличием $\hat{f_1}$ от $f_{opt}$, и называется ошибкой приближения.

Во-вторых, существует ошибка оценивания по данным (estimation error): невозможно получить истинную генеральную совокупность $P_{true}$, а вместо неё есть лишь выборка конечного размера $N$. С учётом такого ограничения задача принимает вид:
$$\hat{f_2} = \arg \min_{f \in \overline{F}} \frac{1}{N} \sum_{i=1}^{N} l(f(x_i), y_i) + \alpha R(f),$$
где $(x_i, y_i)$ — признаки и ответ на $i$-м объекте обучающей выборки, $R(f)$ — регуляризатор, а $\alpha$ — коэффициент, задающий силу регуляризации. Ошибка оценивания — ошибка, вызываемая отличием $\hat{f_2}$ от $\hat{f_1}$.

В-третьих, не всегда удаётся довести поиск минимума из предыдущего пункта до конца, и отсюда возникает ошибка оптимизации (optimization error). Если предположить, что вычислительные ресурсы ограничены и есть всего лишь $T$ итераций, то получится, что решается задача
$$\hat{f_3}^{(T)} = \arg \min_{(T); f \in \overline{F}} \frac{1}{N} \sum_{i=1}^{N} l(f(x_i), y_i) + \alpha R(f),$$
где запись $(T)$ под минимумом обозначает, что минимум ищется за ограниченное число шагов. По аналогии с определением предыдущих ошибок ошибка оптимизации — это ошибка, вызываемая отличием $\hat{f_3}^{(T)}$ от $\hat{f_2}$.

Можно считать, что машинное обучение на больших данных отличается от машинного обучения на маленьких данных тем, что ошибка оптимизации начинает доминировать над ошибкой оценивания по данным.

Подробности есть в статье [Bottou, Bousquet, 2007](https://papers.nips.cc/paper/3323-the-tradeoffs-of-large-scale-learning.pdf).

## Способы автоматического обнаружения выбросов

Выброс (outlier) — объект, считающийся нетипичным для генеральной совокупности, фигурирующей в задаче. Выбросы могут искажать результаты обучения модели, а также результаты оценки качества модели (что создаёт риск неправильного подбора гиперпараметров). Строго говоря, является ли какой-то объект выбросом или нет, должен определять человек, отталкиваясь от своего понимания предметной области и от знаний о том, как именно собирались данные. Тем не менее на практике автоматическое удаление выбросов тоже может существенно улучшить конечный результат.

Способы обнаружения выбросов делятся на две группы в зависимости от того, на какие выбросы они реагируют:

1) Способы, позволяющие выявить одномерные выбросы, то есть выбросы, являющиеся аномальными значениями какого-либо одного признака. Пример одномерного выброса: среди точек на плоскости с координатами (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (100, 6) у последней точки подозрительно большая абсцисса.

2) Способы, позволяющие выявить многомерные выбросы, то есть выбросы, являющиеся аномальными объектами с точки зрения распределения данных. Пример многомерного выброса: среди точек на плоскости с координатами (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (1, 5) последняя точка лежит в стороне от прямой, на которой лежат все остальные точки, но при этом и её абсцисса, и её ордината не выбиваются из диапазона значений остальных точек.

К одномерным способам относятся:
* Стандартизированная оценка (Z-score) — из всех значений признака вычитается эмпирическое среднее этого признака, а полученные разности делятся на оценку стандартного отклонения признака; если получившееся отношение превышает 3, то объект считается выбросом. Порог 3 выбран, потому что в случае нормального распределения вероятность отклониться от среднего больше, чем на 3 стандартных отклонения, составляет 0,03% ([правило трёх сигм](https://ru.wikipedia.org/wiki/%D0%A1%D1%80%D0%B5%D0%B4%D0%BD%D0%B5%D0%BA%D0%B2%D0%B0%D0%B4%D1%80%D0%B0%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%BE%D0%B5_%D0%BE%D1%82%D0%BA%D0%BB%D0%BE%D0%BD%D0%B5%D0%BD%D0%B8%D0%B5#%D0%9F%D1%80%D0%B0%D0%B2%D0%B8%D0%BB%D0%BE_%D1%82%D1%80%D1%91%D1%85_%D1%81%D0%B8%D0%B3%D0%BC)). 
* Устойчивая стандартизированная оценка (robust Z-score) — то же самое, что и стандартизированная оценка, но теперь вычитается медиана, а не среднее, и разности делятся на средний модуль отклонения от медианы, а не на стандартное отклонение. Предпочтительнее в тех ситуациях, где выбросов достаточно много для того, чтобы они могли исказить среднее и стандартное отклонение так, что часть из них будет принята за обычные объекты.
* Метод межквартильного размаха (interquartile range method) — берётся расстояние между 0,75-й и 0,25-й квантилями (так называемый межквартильный размах), а затем выбросами объявляются все наблюдения, отстоящие от медианы более чем на 1,5 межквартильных размаха. Численные параметры приведены в соответствии с классическим вариантом, но, разумеется, их можно поменять (например, вместо двух квартилей взять две другие квантили).

К многомерным способам относятся:
* Способы, где для каждого объекта считается расстояние Махалонобиса от него до некоторого оценённого по данным распределения, принимаемого за генеральную совокупность, а потом некой одномерной процедурой выбросами объявляются сколько-то объектов с наибольшим расстоянием Махалонобиса. Рассмотрим такой подход подробнее. Для многомерного вектора $x$ и многомерного вероятностного распределения $\tau(\mu, S)$ с вектором средних $\mu$ и ковариационной матрицей $S$ "расстоянием" от $x$ до $\tau(\mu, S)$ можно положить величину:
$$D_M(x, \tau(\mu, S)) = \sqrt{(x - \mu)^T S^{-1} (x - \mu)},$$
которая и называется расстоянием Махалонобиса. Интуитивно говоря, расстояние Махалонобиса отличается от евклидова расстояния между $x$ и $\mu$ тем, что каждое направление учитывается с весом, обратно пропорциональным разбросу распределения по этому направлению. В задаче обнаружения выбросов $\tau(\mu, S))$ можно единожды оценить по всем имеющимся данным, если их много, или же для каждого $x$ оценивать по всем объектам кроме $x$, если данных мало.
* Способы, где для каждого объекта считается оценка плотности генеральной совокупности в нём, а затем некой одномерной процедурой выбросами объявляются сколько-то объектов с наиболее низкой оценкой плотности. Оценку плотности можно получить, в частности, методом ядреного сглаживания (kernel density estimate). Как и в предыдущем пункте, если данных мало, то оценку ядерного сглаживания для каждого объекта лучше считать по всем объектам кроме него, а если данных много, то оценку ядерного сглаживания можно подсчитать один раз на всех объектах. Отдельно стоит отметить, что оценки ядерного сглаживания подвержены "проклятью размерности" и работают только в случае, когда размерность данных низкая. Если же размерность высокая, то для применения ядерного сглаживания можно предварительно снизить её, использовав PCA, t-SNE или MDS на отмасштабированных данных, однако как проводить масштабирование — нетривиальный вопрос, вносящий долю субъективности. 

## Автокодировщики с большей размерностью скрытого слоя

Рассмотрим автокодировщик (autoencoder) со входным слоем размерности $l$, одним скрытым слоем размерности $h$ и выходным слоем, размерность которого по определению должна быть равна размерности входного слоя, то есть $l$. Вопрос: могут ли для чего-то пригодиться автокодировщики, где $h > l$?

Без дополнительных модификаций такие автокодировщики не смогут обучаться чему-то нетривиальному, потому что всегда есть вырожденное решение: использовать лишь $l$ нейронов из $h$, чтобы просто передать вход в неизменном виде.

Есть следующие способы заставить скрытый слой выучивать нетривиальное представление:
* К ошибке восстановления, на входном векторе $x$ равной $\Vert x - \mathrm{autoencoder}(x)\Vert_2$ добавить регуляризатор, подталкивающий к разреженности скрытого слоя: например, $L_1$-норму $h$-мерного вектора, получающегося из $x$ в скрытом слое;
* Ко входу подмешивать шум, но при вычислении ошибки восстановления требовать близости к незашумлённому входу, то есть минимизировать математическое ожидание функции, на входном векторе $x$ равной $\Vert x - \mathrm{autoencoder}(x + \varepsilon)\Vert_2$, где $\varepsilon$ — шум.

Полученный автокодировщик можно использовать в следующих целях:
* То, что получается в скрытом слое, можно брать в качестве признакового описания объекта; такие признаки могут содержать в себе более сложные взаимодействия, чем признаки, даваемые автокодировщиком с низким $h$; пригождается это в частичном обучении (semi-supervised learning), ведь объекты без меток тоже влияют на выучиваемый способ порождать признаки;
* До появлении пакетной нормализации и остаточного обучения была популярна "жадная" (послойная от входов к выходам) инициализация весов глубокой однонаправленной нейронной сети, при которой инициализируемый на данный момент слой дополнялся до автокодировщика и обучался задаче автокодировщика, где кодировалось то, что подавал на вход предыдущий слой, — стало быть, если в каком-то месте ширина нейронной сети возрастала, то требовалось обучать автокодировщик, у которого $h > l$.